# Handwritten numbers Recognition

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras 

## Load Data

Extraer datos. Las imágenes son de 28*28 píxeles, por lo tanto el csv tiene 784 columnas más 1 indicando la salida

In [2]:
def extract():
    df_train = pd.read_csv('data/mnist_train.csv', delimiter = ',').to_numpy()
    df_test = pd.read_csv('data/mnist_test.csv', delimiter = ',').to_numpy()
    df = pd.DataFrame(np.concatenate((df_train,df_test)))
    return df

Transformar output a array de 10 y separar de las inputs

In [3]:
def transform(df): 
    y_raw = df.iloc[:, 0].to_numpy()
    x = df.drop(df.columns[0], axis='columns').to_numpy()
    y = []
    for i in range(0,y_raw.shape[0]):
        row=np.zeros(10)
        row[y_raw[i]]=1
        y.append(row)
    y=np.array(y)
    y.reshape(y.shape[0],10)
    return x, y

Creamos los dataset de entrenamiento y test

In [4]:
df = extract()
x_train,y_train = transform(df.iloc[0:60000])
#x_val,y_val = transform(df.iloc[50000:60000])
x_test,y_test = transform(df.iloc[60000:70000])

## Creación de la Red Neuronal

En la red neuronal vamoas a implementar una arquitectura de (784,512,256,128,10), con funciones de activación ReLu en todas las capas menos en la última, en la cual utilizaremos la funcion softmax (https://www.quora.com/Why-is-it-better-to-use-Softmax-function-than-sigmoid-function).<br> Para cada capa se utilizará la funcion Dense, la cual tiene los siguientes parámetros por defecto:
- activation=None,
- use_bias=True,
- kernel_initializer='glorot_uniform',
- bias_initializer='zeros',
- kernel_regularizer=None,
- bias_regularizer=None,
- activity_regularizer=None,
- kernel_constraint=None,
- bias_constraint=None




In [27]:
model = keras.Sequential([
    keras.layers.Dense(784, input_dim=784),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")#Softmax is a sort of Multi Class Sigmoid
])

Vamos a crear y entrenar el modelo, 50 epochs serán suficientes.

In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train, epochs = 50, batch_size=200, verbose=1)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 2s 40us/sample - loss: 4.8348 - accuracy: 0.8692
Epoch 2/50
60000/60000 [==============================] - 2s 36us/sample - loss: 0.6232 - accuracy: 0.9339
Epoch 3/50
60000/60000 [==============================] - 2s 41us/sample - loss: 0.3980 - accuracy: 0.9483
Epoch 4/50
60000/60000 [==============================] - 2s 35us/sample - loss: 0.2907 - accuracy: 0.9557
Epoch 5/50
60000/60000 [==============================] - 2s 36us/sample - loss: 0.2335 - accuracy: 0.9617
Epoch 6/50
60000/60000 [==============================] - 2s 35us/sample - loss: 0.2092 - accuracy: 0.9645
Epoch 7/50
60000/60000 [==============================] - 2s 41us/sample - loss: 0.2160 - accuracy: 0.9646
Epoch 8/50
60000/60000 [==============================] - 3s 47us/sample - loss: 0.2122 - accuracy: 0.9643
Epoch 9/50
60000/60000 [==============================] - 3s 47us/sample - loss: 0.1790 - accuracy: 0.9678
Epoch 10/50
60

Evaluamos el modelo con el acierto de los datos de prueba

In [30]:
tes_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 67us/sample - loss: 0.1687 - accuracy: 0.9634
Test Acc:  0.9633927


## Regularización

In [35]:
from keras import regularizers
modelR = keras.Sequential([
    keras.layers.Dense(784, input_dim=784, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))
])

In [36]:
modelR.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
modelR.fit(x_train, y_train, epochs = 50, batch_size=200, verbose=1)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 3s 49us/sample - loss: 14.6520 - accuracy: 0.8770
Epoch 2/50
60000/60000 [==============================] - 3s 43us/sample - loss: 8.7931 - accuracy: 0.9286
Epoch 3/50
60000/60000 [==============================] - 3s 43us/sample - loss: 6.6997 - accuracy: 0.9459
Epoch 4/50
60000/60000 [==============================] - 3s 43us/sample - loss: 5.1552 - accuracy: 0.9570
Epoch 5/50
60000/60000 [==============================] - 3s 44us/sample - loss: 3.9470 - accuracy: 0.9634
Epoch 6/50
60000/60000 [==============================] - 3s 58us/sample - loss: 3.0012 - accuracy: 0.9686
Epoch 7/50
60000/60000 [==============================] - 3s 57us/sample - loss: 2.2614 - accuracy: 0.9712
Epoch 8/50
60000/60000 [==============================] - 3s 56us/sample - loss: 1.6938 - accuracy: 0.9739
Epoch 9/50
60000/60000 [==============================] - 3s 54us/sample - loss: 1.2680 - accuracy: 0.9737
Epoch 10/50
6

In [37]:
tes_loss, test_acc = modelR.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 87us/sample - loss: 0.2423 - accuracy: 0.9702
Test Acc:  0.97019404
